In [1]:
'''
Zadanie 1:
1. Wczytaj plik nauczyciele.csv, w którym znajdują się dane odnośnie zatrudnienia nauczycieli akademickich w Polsce w
wybranych latach, z podziałem na stanowisko i płeć. Rozróżnione są dodatkowo uniwersystety i szkoły techniczne.
2. Przygotuj dane do dalszej analizy (usuń zbędne wiersze i kolumny, zwróć uwagę na typ danych)
3. Stwórz następujące wykresy:
    • zestawienie nauczycieli akademickich zatrudnionych na dowolnym etacie w latach 2014-2018 z podziałem na województwo
    • zestawienie różnych stanowisk (asystenci, adiunkcji, profesorowie) zatrudnionych w latach 2014-2018 w województwie
śląskim
    • zestawienie nauczycieli akademickich zatrudnionych na uniwersytetach i szkołach technicznych z podziałem na płcie
w województwie śląskim
4. W danych znajdź:
    • wartości dla województwa śląskiego
    • województwa, w których jest zatrudnionych najwięcej i najmniej nauczycieli akademickich
    • województwa, które zatrudniają kobiety ponad średnią

'''
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('nauczyciele.csv',delimiter=';')
df = df.applymap(lambda x:str(x).replace(',','.'))
df.dropna()

def nauczyciele():

    nazwa = df['Nazwa'][1:]
    nauczyciel = df[df.columns[df.columns.str.contains('nauczyciele akademiccy;ogółem')]].apply(
        lambda x: x.str.extract('(\d+)', expand=False).astype(float).sum(), axis=1) [1:]


    plt.plot(range(16), nauczyciel, "o")
    plt.xticks(range(16), nazwa, rotation=70)
    plt.title('Nauczyciele akademiccy 2014-2018')
    plt.ylabel('ilość nauczycieli', fontsize=10)
    plt.show()




def stanowiska():
    nazwa = df[df['Nazwa'] == 'ŚLĄSKIE']
    stanowiska = ['profesorowie', 'asystenci', 'adiunkci']
    liczby = [nazwa[nazwa.columns[nazwa.columns.str.contains(stanowisko)]].apply(
        lambda x: x.str.extract('(\d+)', expand=False).astype(float).sum(), axis=1).item() for stanowisko in stanowiska]

    plt.bar(range(3), liczby, tick_label=stanowiska)
    plt.title('Nauczyciele na poszczególnych stanowiskach w latach 2014-2018')
    plt.ylabel('ilość nauczycieli', fontsize=10)
    plt.show()


def płeć():
    nazwa = df[df['Nazwa'] == 'ŚLĄSKIE']
    szkoly = ['nauczyciele akademiccy;ogółem', 'nauczyciele akademiccy;kobiety']
    liczby = [nazwa[nazwa.columns[nazwa.columns.str.contains(szkola)]].apply(
        lambda x: x.str.extract('(\d+)', expand=False).astype(float).sum(), axis=1).item() for szkola in szkoly]
    liczby[0] -= liczby[1]

    plt.bar(['mężczyźni', 'kobiety'], liczby)
    plt.title('Zatrudnienie nauczycieli akademickich w województwie śląskim (2014-2018)')
    plt.ylabel('liczba nauczycieli', fontsize=10)
    plt.show()

def zad4_1():

    pd.set_option('display.max_rows', 10)

    slaskie = df.loc[df['Nazwa'] == 'ŚLĄSKIE']
    print(slaskie.iloc[0])


def zad4_2():
    a = df.filter(regex='nauczyciele akademiccy;ogółem').apply(pd.to_numeric, errors='coerce').sum(axis=1)
    najwiecej = a.iloc[1:].groupby(df['Nazwa']).sum().nlargest(1).head(1)
    najmniej = a.iloc[1:].groupby(df['Nazwa']).sum().nsmallest(1).head(1)
    print('Najwięcej zatrudnionych jest w województwie:\n', najwiecej.to_string(header=False))
    print('Najmniej zatrudnionych jest w województwie:\n', najmniej.to_string(header=False))

def zad4_3():
    srednia_kobiety = df.filter(regex='nauczyciele akademiccy;kobiety').apply(pd.to_numeric, errors='coerce').mean(axis=1)
    a = srednia_kobiety.iloc[1:][srednia_kobiety > srednia_kobiety.mean()].index.tolist()
    a = df.loc[a, 'Nazwa'].tolist()
    print('Województwa które zatrudniaję kobiety ponad średnią to:\n',a)


In [2]:
'''
Zadanie 2 podpunkt 3.1:
Zestaw 2
1. Wczytaj plik inflacja.xlsx, w którym znajdują się dane dotyczące poziomu inflacji w Polsce (kolejne skoroszyty).
2. Przygotuj dane do dalszej analizy (usuń zbędne wiersze i kolumny, zwróć uwagę na typ danych).
3. Stwórz następujące wykresy:
    • zestawienie roku 2017, 2018 i 2019 w odniesieniu do grudnia poprzeniego roku (Table 1)
    • zestawienie października roku 2017, 2018 i 2019 w odniesieniu do poprzeniego miesiąca danego roku (Table 2)
    • zestawienie roku 2017, 2018 i 2019 w odniesieniu do analogicznego miesiąca poprzedniego roku (Table 3)
4. W danych znajdź:
    • wartości największa i najmniejszą w każdej kategorii
    • wyznacz nawyższe wartości inflacji w poszczególnych latach
    • wyznacz średnią wartośc inflacji w poszczególnych latach

'''

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_excel("inflacja.xlsx", sheet_name=None)

df = df['Table 1']
df.iat[4, 2] = ''
df.iat[5, 2] = 'Rok'
df = df[5:39]
df = df.iloc[:, 2:15]
df.columns = df.iloc[0]
df = df[1:]
df = df.apply(pd.to_numeric, errors='coerce')

R2017 = df[df['Rok'] == 2017]
R2018 = df[df['Rok'] == 2018]
R2019 = df[df['Rok'] == 2019]

x = np.arange(len(R2017.columns[1:]))
width = 0.25

plt.bar(x, R2017.values.flatten()[1:], width=width, label='2017')
plt.bar(x + width, R2018.values.flatten()[1:], width=width, label='2018')
plt.bar(x + 2*width, R2019.values.flatten()[1:], width=width, label='2019')
plt.axhline(100, color = 'red')

plt.xlabel('Miesiąc')
plt.ylabel('Inflacja')
plt.title('Zestawienie roku 2017, 2018 i 2019')
plt.xticks(x + width, R2017.columns[1:])
plt.ylim(99, 103)
plt.legend(labels = ['Grudzień zeszł rok','2017','2018','2019'])
plt.show()


In [3]:
# Zadanie 2 podpunkt 3.2:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_excel("inflacja.xlsx", sheet_name='Table 2', header=None)
df = df.iloc[:,1:15]
df.columns = df.iloc[39]
df.iat[38, 0] = ''
df.iat[39, 0] = 'Rok'
df['Rok'] = df['Rok'].astype(str)

rok = ['2017', '2018', '2019']
a = df.loc[df['Rok'].isin(rok), 'X'].values # Wartość dla października


plt.bar(rok, a)
plt.axhline(100, color = 'red')

plt.xlabel('Rok')
plt.ylabel('Wartość')
plt.title('Zestawienie wartości dla lat 2017, 2018 i 2019')
plt.ylim(99, 103)
plt.legend(labels = ['Poprzedni miesiąc danego roku'])
plt.show()





<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'inflacja.xlsx'

In [4]:
# Zadanie 2 podpunkt 3.3:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_excel("inflacja.xlsx", sheet_name='Table 3', header=None)
df = df.iloc[:35,1:15]
df.columns = df.iloc[34]
df.iat[33, 0] = ''
df.iat[34, 0] = 'Rok'
df['Rok'] = df['Rok'].astype(str)
df = df.apply(pd.to_numeric, errors='coerce')

R2017 = df[df['Rok'] == 2017]
R2018 = df[df['Rok'] == 2018]
R2019 = df[df['Rok'] == 2019]

x = np.arange(len(R2017.columns[1:]))
width = 0.25

plt.bar(x, R2017.values.flatten()[1:], width=width, label='2017')
plt.bar(x + width, R2018.values.flatten()[1:], width=width, label='2018')
plt.bar(x + 2*width, R2019.values.flatten()[1:], width=width, label='2019')
plt.axhline(100, color = 'red')

plt.xlabel('Miesiąc')
plt.ylabel('Inflacja')
plt.title('Zestawienie roku 2017, 2018 i 2019')
plt.xticks(x + width, R2017.columns[1:])
plt.ylim(99, 103)
plt.legend(labels = ['Grudzień zeszł rok','2017','2018','2019'])
plt.show()

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: 'inflacja.xlsx'

In [ ]:
# Zadanie 2 podpunkt 4:
import pandas as pd

df = pd.read_excel("inflacja.xlsx", sheet_name='Table 4', header=None)

df = df.iloc[1:,2:15]
df.columns = df.iloc[0]
df.iat[0, 0] = 'Rok'


df.iloc[2:, 1:] = df.iloc[2:, 1:].apply(pd.to_numeric, errors='coerce')
df['Rok'] = df['Rok'].astype(str)

def zad4_1():
    pk = 'I' #Pierwsza kolumna
    kk = 'XII' #Końcowa kolumna
    df = df.loc[:,pk:kk]

    maks = df.max() # Największa wartość inflacji w poszczególnych miesiącach
    mini = df.min() #Najmniejsza wartość inflacji w poszczególnych miesiącach

    print('Największa wartość inflacji w poszczególnych miesiącach to: ',maks)
    print('Najmniejsza wartość inflacji w poszczególnych miesiącach to: ',mini)

def zad4_2():
    rok = df.iloc[:, 0].astype(str)
    maks_rok = df.iloc[:, 1:].max(axis=1)
    maks_rok.index = rok

    print("Najwyższe wartości inflacji w poszczególnych latach:", maks_rok)


def zad4_3():
    sr = df.iloc[:, 0].astype(str)
    srednia = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').mean(axis=1)
    srednia.index = sr

    print("Średnia wartość inflacji w poszczególnych latach to: ", srednia)